## Overview

In this notebook you will find ...
1. A recap of the main result of the Rademacher complexity in the context of statistical learning theory
2. A playground for applying and testing those (theoretical) results to the hypothesis class of ridge regression, 

The purpose of this notebook is to empirically validate all the theoretical results as well as getting a feeling how the Rademacher complexity behaves.

## Recap of mathematical results

The main result about the Rademacher complexity reads as follows:

$$R(\hat{h})\le R(h^*)+2\mathcal{R}_n(\mathcal{L})+K\left(\frac{ln(1/\delta)}{2n}\right)^{1/2}$$

with probability at least $1-\delta$ where $R$ denotes the risk, $\hat{h}=\mathcal{A}(D_n)$ is the parameter obtained by (our) algorithm, $R(h^*)$ is the minimal risk among all hypothesis, $\mathcal{R}_n(\mathcal{L})$ denotes the Rademacher complexity with respect to the composition of loss function with the underlying hypothesis class, $K$ is the supremum of the loss function restricted to the image of the hypothesis class. 

Furthermore, the empirical Rademacher complexity defined as 
$$\hat{\mathcal{R}}_n(\mathcal{L})=\mathbb{E}_{\epsilon}\left[sup_{h\in \mathcal{H}}\frac{1}{n}\sum_{i=1,...,n}\epsilon_iL\circ h(Z_i)\right]$$
satisfies 
$\mathbb{E}\left[\hat{\mathcal{R}}_n(\mathcal{L})\right]=\mathcal{R}_n(\mathcal{L})$ and serves, therefore, as an approximate of the Rademacher complexity.

## Import of required modules

In [ ]:
import numpy as np
from modules.parametric_model import PenalizedLinearModel
import matplotlib.pyplot as plt
from ipywidgets import *

## Generating training data

We first generate some training data (i.e. sample the random variables $Z$) by defining
$$Z=(X,f(X)+\epsilon)$$
where $X$ is uniformly distributed on the interval $[0,1]$, $\epsilon\sim \mathcal{U}[-1/2,1/2]$ is a uniformly distributed noise (independent of $X$) and $f$ is the function defined below.

In [ ]:
true_beta = 4
def linear_function(x):
    return true_beta*x

In [ ]:
# training data (X,f(X)) with X uniform on [0,1]
def generate_training_data(number_samples: int):
    return [np.array([x, linear_function(x)+np.random.uniform(-1/2,1/2)]) for x in np.random.uniform(0, 1, number_samples)]

In [ ]:
sample_size = 10
training_data = generate_training_data(sample_size)

## Hypothesis class: Ridge Regression

In this notebook we deal with Ridge regression, i.e. our hypothesis class looks as follows:
$$\mathcal{H}=\{x\mapsto \beta^Tx\colon ||\beta||_q\le M\}$$
for some real number $M$ and positive number $q$.

In the following we set 
$$q=2,M=3$$
but you are free to change the values.

In [ ]:
q_of_q_norm = 2
M = 3
# Initialize ridge regression model
model = PenalizedLinearModel(
    training_data=training_data, maximum_beta=M, q_of_q_norm=q_of_q_norm)

We use the ERM (expected risk minimizer) algorithm in order to obtain a hypothesis from our samples.
This is already implemented in the train method of the above model.

In [ ]:
model.train(initial_guess=np.array([0]), max_iter_training=100)

Let us visualize the training data and the trained model.

In [ ]:
# Visualize model
x = np.array(training_data).T[0]
y = np.array(training_data).T[1]

fig, ax = plt.subplots(layout='constrained')
ax.scatter(x, y, label="Training data")
ax.plot([0,1], [model(parameter=model.trained_parameter, x=xi)
        for xi in [0,1]], label=f"Trained function; $\\beta={model.trained_parameter[0]:.2f}$")
ax.plot([0,1], [linear_function(xi)
        for xi in [0,1]], label=f"True function; $\\beta={true_beta:.2f}$")
ax.legend()

## Empirical Rademacher complexity

We wish to calculate the empirical Rademacher complexity 
$$\hat{\mathcal{R}}_n(\mathcal{L})=\mathbb{E}_{\epsilon}(sup_{h\in \mathcal{H}}\frac{1}{n}\sum_{i=1,...,n}\epsilon_iL\circ h(Z_i)).$$
However, since we are taking an expected value, we *approximate* the empirical Rademacher complexity by the following algorithm

```
    for $k=1,...,K$ 
    1. simulate Rademacher variables 
```
$$\epsilon_1,...,\epsilon_n$$
```
    2. compute 
```
$$\mathcal{R}_k= max_{h\in \mathcal{H}}\frac{1}{n}\sum_{i=1}^n\epsilon_iL\circ h(Z_i)$$

Since $$\frac{1}{K}\sum_{k=1}^K\mathcal{R}_k\overset{K\to \infty}\to \hat{\mathcal{R}}_n$$
almost surely, $\frac{1}{K}\sum_{k=1}^K\mathcal{R}_k$ serves as an approximation of the empirical Rademacher complexity

**Exercise:** Implement the empirical Rademacher complexity according to the above algorithm.

**Note:** The loss function (sqare loss) is alread implemented in the model (method: model.loss_function()) and takes two inputs $z_1, z_2$ where $z=(z_1,z_2)=(x,y)$ (i.e. as the form of the training data). 


For example: 

In [ ]:
model.loss_function(training_data[0], training_data[1])

**Bonus Exercise:** Implement the empirical Rademacher complexity in the function below. The function should return the value as a float.

In [ ]:
# def calculate_empirical_rademacher_complexity(K: int,
#                                              hypothesis_class: PenalizedLinearModel,
#                                              max_iter_maximization: float = 1000) -> float:
#    raise NotImplementedError

In [ ]:
from modules.empirical_rademacher import calculate_empirical_rademacher_complexity  

Let us calculate the approximation of the empirical Rademacher complexity.

In [ ]:
K = 100
print(
    f"The approximated empirical rademacher complexity with K={K} is {calculate_empirical_rademacher_complexity(K=K,hypothesis_class=model,)}.")

### Verification

Recall that the Rademacher complexity serves as a bound 
$$\sup_{h\in \mathcal{H}}(R(h)-R_n(h))\le 2\mathcal{R}_n(\mathcal{L})+K\sqrt{\frac{ln(1/\delta)}{2n}}$$
with probability at least $1-\delta$.

Let us verify this empirically by visualizing $R(h)-R_n(h)$ and the upper right side for all $h$.

We calculate
$$K=sup_{z,l}|l(z)|=sup_{\beta\in [-3,3]}(\beta_{true}-\beta+1/2)^2=(15/2)^2.$$

In our setup we can explicitely calculate the risk (since we know the underlying distribution of the samples):
$$R(h)=\int_{0}^1(4x-\beta x)^2dx+var(\epsilon)=\frac{(4-\beta)^2}{3}+1/3$$

Observe that in our setup $||\beta||_q\le M$ reads $\beta\in [-3,3]$. 

In [ ]:
def calculate_risk_of_hypothesis(beta):
    return ((true_beta-beta)**2)/3+1/3

In particular, the bound yields 
$$R(h)\le R_n(h)+2\mathcal{R}_n(\mathcal{L})+K\sqrt{\frac{ln(1/\delta)}{2n}}$$
for every $h\in \mathcal{H}$ with probability at least $1-\delta$.
In other words, based on the empirical risk $R_n(h)$ (which we can calculate) we can bound the risk $R(h)$ (which we _cannot_ calculate) of every hypothesis $h\in \mathcal{H}$. Let us visualize that!

In [ ]:
sample_size = 1000
x = np.arange(-M, M, 0.1)
y = calculate_risk_of_hypothesis(
    x)
training_data = generate_training_data(sample_size)
model = PenalizedLinearModel(
    training_data=training_data, maximum_beta=M, q_of_q_norm=q_of_q_norm)
model.train(initial_guess=0)
delta = 0.05 # 95% confidence
rademacher = calculate_empirical_rademacher_complexity(
    K=100, hypothesis_class=model)
sup_L = (np.max([true_beta-M,true_beta+M])+1/2)**2
bound = 2*rademacher+sup_L*np.sqrt(np.log(1/delta)/(2*sample_size))+np.array([model.empirical_risk(xi) for xi in x])

In [ ]:
fig, ax = plt.subplots(layout='constrained')
# ax.set_ylim(ymin=0, ymax=20)
ax.plot(x, y, label="True risk")
ax.plot(x, bound, label="Bound")
ax.set_xlabel('Parameter beta')
ax.set_ylabel('Value of risk/bound')
ax.legend()

**Exercise:** Vary over the sample size and see how the bound decreases.

**Bonus:** Even for a the large sample size ($n=1000$) the bound is not tight, i.e. the risk of the hypothesis is much smaller than the bound. Why is that?


### Optional: Benchmark empirical Rademacher complexity

Since $\frac{1}{K}\sum_{k=1}^K\mathcal{R}_k$ is only an approximate of the empirical Rademacher complexity (and is a Random variable) let us estimate its mean and standard deviation.

In [ ]:
%%capture
result = [calculate_empirical_rademacher_complexity(
    K=K, hypothesis_class=model,) for _ in range(10)]

In [ ]:
print(f"Estimated mean: {np.mean(result)} and estimated std: {np.std(result)}")

**Exercise:**
Find some $K$ such that the standard deviation is below 10% of the mean.

Let us further look how the approximation changes with the sample size.

In [ ]:
%%capture
sample_sizes = [100, 200, 500, 1000]
rademachers = []
for sample_size in sample_sizes:
    training_data = generate_training_data(number_samples=sample_size)
    model = PenalizedLinearModel(
        training_data=training_data, maximum_beta=M, q_of_q_norm=q_of_q_norm)
    model.train(initial_guess=np.array([0]), max_iter_training=100)
    rademachers.append(calculate_empirical_rademacher_complexity(
        K=200, hypothesis_class=model))

In [ ]:
# Create a figure containing a single axes.
fig, ax = plt.subplots(layout='constrained')
ax.plot(sample_sizes, rademachers)
ax.legend()
ax.set_xlabel('Sample size')  # Add an x-label to the axes.
# Add a y-label to the axes.
ax.set_ylabel('Approximation empirical Rademacher')
# ax.set_ylim(ymin=0)

### Comparison to theoretical bounds

In the lecture we have established a bound of the Rademacher complexity *with respect to the **hypothesis class***, i.e.
$$\mathcal{R}_n(\mathcal{H})\leq M\sqrt{\frac{\mathbb{E}\left[||Z||_2^2\right]}{n}}.$$
In our case, we can calculate 
$$\mathbb{E}\left[||Z||_2^2\right]=\int_{0}^1(x^2+(4x)^2)dx+var(\epsilon)=17/3+1/3=6$$
and, therefore, 
$$\mathcal{R}_n(\mathcal{H})\leq 3\sqrt{6/n}.$$

Applying Talagrands contraction lemma, we obtain
$$\mathcal{R}_n(\mathcal{L})\leq 2\sqrt{B}\mathcal{R}_n(\mathcal{H})$$
where $B$ is the supremum of the loss. 

In [ ]:
def calculate_theoretical_bound(n):
    return 2*(sup_L)**(1/2)*M*(6/n)**(1/2)

In [ ]:
ax.plot(sample_sizes, [calculate_theoretical_bound(sample_size)
        for sample_size in sample_sizes], label="Bound")
ax.set_ylim(ymin=0)
ax.legend()
fig

## Bound of risk

Recall that we are interested in a bound of the empirical loss and that the Rademacher complexity provides such a bound:
$$R(\hat{h})\le R(h_0)+2\mathcal{R}_n(\mathcal{L})+K\left(\frac{ln(1/\delta)}{2n}\right)^{1/2}$$
with probability at least $1-\delta$.

In our setup we can explicitly calculate the risk (since we know the underlying distribution of the samples):
$$R(h)=\frac{(4-\beta)^2}{3}+1/3$$

Therefore, we can calculate
$$R(h_0)=\frac{2}{3},\beta_0=3.$$

Above we calculated
$$K=(15/2)^2.$$

We obtain 
$$R(\hat{h})\le \frac{1}{3}+2\mathcal{R}_n(\mathcal{L})+K^2\left(\frac{ln(1/\delta)}{2n}\right)^{1/2}.$$

Let us empirically verify that bound by using the approximation of the empirical rademacher instead of $\mathcal{R}_n(\mathcal{L})$ (and neglecting the approximation error). 

In [ ]:
bound = calculate_risk_of_hypothesis(3)+2*calculate_empirical_rademacher_complexity(
    K=100, hypothesis_class=model)+(15/2)**2*(np.log(1/delta)/(2*len(training_data)))**(1/2)

Now, let us convince that this is in fact an upper bound of the risk of the hypothesis calculated by our algorithm:

In [ ]:
print(f"The calculated bound is {bound}")
print(
    f"The calculated risk is {calculate_risk_of_hypothesis(model.trained_parameter)[0]}")

Let us at last look if this is also a bound of the risk of the other hypothesis.

**Exercise:** What do you expect? Does the bound above really bounds all risks? If not, why?

Observe that in our setup $||\beta||_q\le M$ reads $\beta\in [-3,3]$.
Lets visualize the risk of all those hypothesis.

In [ ]:
x = np.arange(-3, 3, 0.1)
y = calculate_risk_of_hypothesis(x)

fig, ax = plt.subplots(layout='constrained')
ax.plot(x, y, label="Risk of respective hypothesis")
ax.plot(x, [bound for _, _ in enumerate(x)], label="Bound")
ax.set_xlabel('Parameter beta')
ax.set_ylabel('Value of risk/bound')
ax.legend()